Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 01. Logging APIs
This notebook showcase various ways to use the Azure Machine Learning service run logging APIs, and view the results in the Azure portal.

## Prerequisites
Make sure you go through the [configuration notebook](../../../configuration.ipynb) first if you haven't. Also make sure you have tqdm and matplotlib installed in the current kernel.

```
(myenv) $ conda install -y tqdm matplotlib
```

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
from azureml.core import Experiment, Run, Workspace
import azureml.core
import numpy as np

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Set experiment
Create a new experiment (or get the one with such name).

In [ ]:
exp = Experiment(workspace=ws, name='logging-api-test')

## Log metrics
We will start a run, and use the various logging APIs to record different types of metrics during the run.

In [ ]:
from tqdm import tqdm

# start logging for the run
run = exp.start_logging()

# log a string value
run.log(name='Name', value='Logging API run')

# log a numerical value
run.log(name='Magic Number', value=42)

# Log a list of values. Note this will generate a single-variable line chart.
run.log_list(name='Fibonacci', value=[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89])

# create a dictionary to hold a table of values
sines = {}
sines['angle'] = []
sines['sine'] = []

for i in tqdm(range(-10, 10)):
    # log a metric value repeatedly, this will generate a single-variable line chart.
    run.log(name='Sigmoid', value=1 / (1 + np.exp(-i)))
    angle = i / 2.0
    
    # log a 2 (or more) values as a metric repeatedly. This will generate a 2-variable line chart if you have 2 numerical columns.
    run.log_row(name='Cosine Wave', angle=angle, cos=np.cos(angle))
        
    sines['angle'].append(angle)
    sines['sine'].append(np.sin(angle))

# log a dictionary as a table, this will generate a 2-variable chart if you have 2 numerical columns
run.log_table(name='Sine Wave', value=sines)

run.complete()

Even after the run is marked completed, you can still log things.

## Log an image
This is how to log a _matplotlib_ pyplot object.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
angle = np.linspace(-3, 3, 50)
plt.plot(angle, np.tanh(angle), label='tanh')
plt.legend(fontsize=12)
plt.title('Hyperbolic Tangent', fontsize=16)
plt.grid(True)

run.log_image(name='Hyperbolic Tangent', plot=plt)

## Upload a file

You can also upload an abitrary file. First, let's create a dummy file locally.

In [ ]:
%%writefile myfile.txt

This is a dummy file.

Now let's upload this file into the run record as a run artifact, and display the properties after the upload.

In [ ]:
props = run.upload_file(name='myfile_in_the_cloud.txt', path_or_stream='./myfile.txt')
props.serialize()

## Examine the run

Now let's take a look at the run detail page in Azure portal. Make sure you checkout the various charts and plots generated/uploaded.

In [ ]:
run

You can get all the metrics in that run back.

In [ ]:
run.get_metrics()

You can also see the files uploaded for this run.

In [ ]:
run.get_file_names()

You can also download all the files locally.

In [ ]:
import os
os.makedirs('files', exist_ok=True)

for f in run.get_file_names():
    dest = os.path.join('files', f.split('/')[-1])
    print('Downloading file {} to {}...'.format(f, dest))
    run.download_file(f, dest)   